In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
##Remove the useless data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [8]:
##Encode Categorical Variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [10]:
#OneHot Encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [11]:
#Combine the onehot encoded columns with original data
data= pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
#Split the data into features and target
X= data.drop('EstimatedSalary',axis=1)
y= data['EstimatedSalary']

In [20]:
##Split the data in training and testing set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
##Scale down these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [23]:
#Save the encoders and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Regression Problem Statement

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [27]:
##Building ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##Hidden Layer connected with input layer
    Dense(32,activation='relu'), ##HL2
    Dense(1) ##Output Layer
])

##Compile the Model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

#Set Up Tensorboard
log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [29]:
##Set Up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [30]:
##Train The Model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 100365.8594 - mae: 100365.8594 - val_loss: 98475.4609 - val_mae: 98475.4609
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99499.6250 - mae: 99499.6250 - val_loss: 96744.7891 - val_mae: 96744.7891
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96517.9922 - mae: 96517.9922 - val_loss: 92393.2188 - val_mae: 92393.2188
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 90713.8125 - mae: 90713.8125 - val_loss: 85244.0391 - val_mae: 85244.0391
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 82387.7734 - mae: 82387.7734 - val_loss: 76239.8828 - val_mae: 76239.8828
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 72924.2266 - mae: 72924.2266 - val_loss: 67310.6406 - val_mae: 67310.6406
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 64183.6562 - mae: 64183.656

In [31]:
##Load Tensorboard Session
%load_ext tensorboard

In [34]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 4004), started 0:00:07 ago. (Use '!kill 4004' to kill it.)

In [35]:
## Evaluate Model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50296.1719 - mae: 50296.1719
Test MAE: 50296.171875


In [36]:
model.save('regression_model.h5')

d:\Projects\End2EndDL_ann\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
